In [ ]:
from clearml import Task, Logger
import pickle
import os
import configparser
import json
from tqdm import tqdm
import io
from contextlib import redirect_stdout
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
from task_processing import read_local_pkl

In [ ]:
def get_credentials(dir_, file):
    if file is not None:
        config = configparser.ConfigParser()
        config.read(f'{dir_}/{file}.conf')

        # Get the values of the different settings
        api_server = config.get('settings', 'api_server')
        web_server = config.get('settings', 'web_server')
        files_server = config.get('settings', 'files_server')

        # Parse the credentials JSON string
        credentials_json = config.get('settings', 'credentials')
        credentials = json.loads(credentials_json)
        access_key = credentials['access_key']
        secret_key = credentials['secret_key']
        return api_server, web_server, files_server, access_key, secret_key
    else:
        return None, None, None, None, None

In [ ]:
def write2server(root, files, project_name='Test', force=False):
    if all(['data_dict.pkl' in files, 'params.pkl' in files, 'scalars.pkl' in files]):
        trap = io.StringIO()
        with redirect_stdout(trap):
            data_dict, params, scalars = read_local_pkl(root)
            if 'imported' not in data_dict['system_tags'] or force:
                data_dict['system_tags'] = list(set(data_dict['system_tags'] + ['imported']))
                task = Task.init(project_name=project_name, task_name=data_dict['name'])
                task.add_tags(data_dict['tags'] + ['imported'])
                _ = task.connect(params['Args'])
                task.set_comment(data_dict['comment'])
                for graph in scalars:
                    for scalar in scalars[graph]:
                        for val_x, val_y in zip(scalars[graph][scalar]['x'], scalars[graph][scalar]['y']):
                            Logger.current_logger().report_scalar(graph, scalar, val_y, val_x)
                if 'archived' in data_dict['system_tags']:
                    task.set_archived(True)
                task.close()
                with open(f"{root}/data_dict.pkl", 'wb') as f:
                    pickle.dump(data_dict, f)
            

In [ ]:
configuration = {
    'config_dir': '../configs',
    'server': 'local_server',
    'force': False,
    'project_name': "AI Fairness"
}

In [ ]:
api_server, web_server, files_server, access_key, secret_key = get_credentials(configuration['config_dir'], configuration['server'])
if api_server is not None:
    Task.set_credentials(
        api_host=api_server, web_host=web_server, files_host=files_server,
        key=access_key, secret=secret_key
     )
else:
    print('Use default clearml.conf')

In [ ]:
for root, dirs, files in tqdm(os.walk('../saved_tasks'), total=len(list(os.walk('../saved_tasks')))):
    write2server(root, files, configuration['project_name'], configuration['force'])